# Test 13: Jet and MET
Test `jet_MET` function which collects the information of $\Delta\phi$ between jet and MET for each event by using SVJ with CKKW-L and without decay data.

## 1. Import Packages

In [1]:
# The Python Standard Library
import os
import sys
import time
import datetime
import glob
import multiprocessing as mp

# The Third-Party Library
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import prettytable
import uproot
import pyjet
import importlib

# My Packages
import myhep.particle_information_v2 as mypInfo_v2
import myhep.analytical_function_v2 as myaFun_v2
import myhep.analysis_v3 as myAnal_v3
# import myhep.particleinfo_v1 as mypiv1
# import myhep.particlefun_v1 as myafv1

# increase figure showing resolution
%config InlineBackend.figure_format = 'retina'

## 2. Import .root File and Load the Data via class
Skip:  
2-1. Check the number of events for each branch  
2-2. Define mass quantities

In [2]:
INPUT_FILE = '/youwei_u3/svj_data_master/scheme_1/root/ckkwl_wo.root'

DATA = uproot.open(INPUT_FILE)['Delphes;1']
GP = mypInfo_v2.classGenParticle(DATA)
Jet = mypInfo_v2.classJet(DATA)
Event = mypInfo_v2.classEvent(DATA)

## 3. Analyze the Dark Sector in the Parton and Truth Levels
Skip

## 4. Jet Clustering

### 4-1. Select stable final state particles without/with filtering out dark sector

In [3]:
SFSP, SFSP_filterDM = myAnal_v3.selectStableFinalStateParticle(
    GP, filter=[51, -51, 53, -53, 4900211, -4900211, 4900213, -4900213,
                4900101, -4900101, 4900021,
                12, -12, 14, -14, 16, -16])

The PID of dark matter are [51, -51, 53, -53, 4900211, -4900211, 4900213, -4900213, 4900101, -4900101, 4900021, 12, -12, 14, -14, 16, -16].
19373 events are stable final state.
19373 events are stable final state without DM.


### 4-2. Let's do the jet clustering!!

In [4]:
R, JetClusteringAlgorithm, pTmin_pyjet = 0.4, -1, 0

PseudoJet = myAnal_v3.jetClustering_v1(SFSP, R=R,
                                       p=JetClusteringAlgorithm,
                                       pTmin=pTmin_pyjet)
PseudoJet_filterDM = myAnal_v3.jetClustering_v1(SFSP_filterDM, R=R,
                                                p=JetClusteringAlgorithm,
                                                pTmin=pTmin_pyjet)
print('Done')

Done


## 5. Analyze the Jet in the Truth Level

### 5-1. Preselection version 1

In [5]:
presel_bef, presel_pt, presel_pt_eta, presel_idx = myAnal_v3.preselection_v1(PseudoJet_filterDM, pT_min=20, eta_max=2.5)

19373 events before preselection
19373 events after pT preselection
19373 events after pT & eta preselections
--------------------------------------------------------------------------------
0 events without PseudoJet before preselection
364 events without PseudoJet after pT preselection
526 events without PseudoJet after pT & eta preselections


### 5-2. MET

In [6]:
arr_MET, df_MET = myAnal_v3.MET_visParticles_v1(SFSP_filterDM)

19373 events in MET data.


### 5-3. Jet and MET

In [7]:
df_jet_MET = myAnal_v3.jet_MET(presel_pt_eta, arr_MET)

19373 events.
Azimuthal angle difference between 4 leading jets and MET, and minimum azimuthal angle between all jets and MET.


In [8]:
df_jet_MET.head()

,N_jet,Dphi_j1_MET,Dphi_j2_MET,Dphi_j3_MET,Dphi_j4_MET,min_Dphi_j_MET
0,2.0,2.964638,3.070738,-999.000000,-999.000000,2.964638
1,3.0,1.770399,2.603022,1.769629,-999.000000,1.769629
2,1.0,3.140019,-999.000000,-999.000000,-999.000000,3.140019
3,1.0,2.997144,-999.000000,-999.000000,-999.000000,2.997144
4,4.0,2.897558,2.945442,0.079480,0.844717,0.079480


##### Conclusion: Well done!!

In [9]:
# !!! test 'jet_MET' function !!!
# 
_jet_met = []
for i in range(85, 88): # test 0~3 or 85~88
    n_jet = presel_pt_eta[i].shape[0]
    pt, eta = presel_pt_eta[i]['pT'], presel_pt_eta[i]['eta']
    phi, mass = presel_pt_eta[i]['phi'], presel_pt_eta[i]['mass']
    met, met_phi = arr_MET[i]['MET'], arr_MET[i]['phi']
    j_met = np.array([n_jet])
    if n_jet != 0:
        print('There is the jet.')
        Dphi = np.absolute(phi - met_phi)
    else:
        print('There is NO the jet.')
        Dphi = np.full(4, -999, dtype=np.float64)
    print(f'event {i}: phi = {phi}, MET = {met}, MET_phi = {met_phi}')
    print(f'         Dphi_1 = {Dphi}')
    Dphi[Dphi > np.pi] = 2*np.pi - Dphi[Dphi > np.pi]
    min_Dphi = np.array([np.amin(Dphi)])
    print(f'         Dphi_2 = {Dphi}, min[Dphi] = {min_Dphi}')
    if n_jet < 4:
        diff = 4 - n_jet
        arr_n999 = np.full(diff, -999, dtype=np.float64)
        Dphi = np.concatenate((Dphi, arr_n999), axis=None)
    print(f'         Dphi_3 = {Dphi}')
    j_met = np.concatenate((j_met, Dphi[:4], min_Dphi), axis=None)
    print(f'         j_MET = {j_met}')
    _jet_met.append(j_met)
    print('-'*80)
arr_jet_met = np.stack(_jet_met, axis=0)
arr_jet_met

There is the jet.
event 85: phi = [-1.7157056   0.36653449 -1.79523995  1.73606785 -0.64120926 -2.47622644], MET = 178.89822936222123, MET_phi = 2.0723682399119148
         Dphi_1 = [3.78807384 1.70583375 3.86760819 0.33630039 2.7135775  4.54859468]
         Dphi_2 = [2.49511146 1.70583375 2.41557711 0.33630039 2.7135775  1.73459063], min[Dphi] = [0.33630039]
         Dphi_3 = [2.49511146 1.70583375 2.41557711 0.33630039 2.7135775  1.73459063]
         j_MET = [6.         2.49511146 1.70583375 2.41557711 0.33630039 0.33630039]
--------------------------------------------------------------------------------
There is NO the jet.
event 86: phi = [], MET = 11.642289591890822, MET_phi = 2.4409867632787035
         Dphi_1 = [-999. -999. -999. -999.]
         Dphi_2 = [-999. -999. -999. -999.], min[Dphi] = [-999.]
         Dphi_3 = [-999. -999. -999. -999. -999. -999. -999. -999.]
         j_MET = [   0. -999. -999. -999. -999. -999.]
----------------------------------------------------------

array([[ 6.00000000e+00,  2.49511146e+00,  1.70583375e+00,
         2.41557711e+00,  3.36300394e-01,  3.36300394e-01],
       [ 0.00000000e+00, -9.99000000e+02, -9.99000000e+02,
        -9.99000000e+02, -9.99000000e+02, -9.99000000e+02],
       [ 2.00000000e+00,  3.03885763e+00,  1.06829695e+00,
        -9.99000000e+02, -9.99000000e+02,  1.06829695e+00]])

## Test

### Test A: basic

In [10]:
print(presel_pt_eta[0].dtype)
presel_pt_eta[0]

[('pT', '<f8'), ('eta', '<f8'), ('phi', '<f8'), ('mass', '<f8')]


array([(87.39944212, 0.18768543, -0.16260718, 13.97055667),
       (64.21380757, 0.61326145, -0.41041687, 11.67831571)],
      dtype=[('pT', '<f8'), ('eta', '<f8'), ('phi', '<f8'), ('mass', '<f8')])

In [11]:
print(presel_pt_eta[0]['pT'].dtype)
presel_pt_eta[0]['pT']

float64


array([87.39944212, 64.21380757])

In [12]:
print(arr_MET[:5].dtype)
arr_MET[:5]

[('MET', '<f8'), ('phi', '<f8'), ('METx', '<f8'), ('METy', '<f8')]


array([(155.78999264,  2.80203073, -146.89448818,  51.88960539),
       (158.13608629, -0.31722873,  150.24565215, -49.32814406),
       (201.62627625, -0.37612596,  187.53151556, -74.06136607),
       (105.30642437,  2.74217592,  -97.01754218,  40.95167303),
       (204.68373461,  0.53197955,  176.397395  , 103.8238424 )],
      dtype=[('MET', '<f8'), ('phi', '<f8'), ('METx', '<f8'), ('METy', '<f8')])

In [13]:
print(arr_MET['MET'].dtype)
arr_MET['MET'][:5]

float64


array([155.78999264, 158.13608629, 201.62627625, 105.30642437,
       204.68373461])

In [14]:
# find 0-jet events
for i in range(100):
    if presel_pt_eta[i].shape[0] == 0:
        print(i)

28
86


In [15]:
presel_pt_eta[28]

array([],
      dtype=[('pT', '<f8'), ('eta', '<f8'), ('phi', '<f8'), ('mass', '<f8')])

In [16]:
presel_pt_eta[28]['pT']

array([], dtype=float64)

In [17]:
presel_pt_eta[28]['phi']

array([], dtype=float64)

In [18]:
print(f"pT = {presel_pt_eta[28]['pT']}")
print(f"eta = {presel_pt_eta[28]['eta']}")
print(f"phi = {presel_pt_eta[28]['phi']}")
print(f"mass = {presel_pt_eta[28]['mass']}")

pT = []
eta = []
phi = []
mass = []


### Test B: stack all events

In [19]:
switch = False
_jet_met = []
a, b = np.array([1]), np.array([10])
MET_phi = np.array([(100, 4), (100, 1), (200, 2), (300, 30)],
                   dtype=[('MET', '<f8'), ('phi', '<f8')])
print(f'MET_phi = {MET_phi}\n', '-'*80)
for i, element in enumerate([1, 2, 3, 6]):
    print(f'switch = {switch}')
    if switch == False:
        pt, phi = np.array([]), np.array([])
    else:
        a, b = np.append(a, [element]), np.append(b, [10 * element + element])
        pt, phi = a, b
    switch = True
    met, met_phi = MET_phi[i]['MET'], MET_phi[i]['phi']
    n_pt = pt.shape[0]
    j_met = np.array([n_pt])
    if n_pt != 0:
        print('There is jet.')
        Dphi = np.absolute(phi - met_phi)
    else:
        print('There is NO jet.')
        Dphi = np.full(4, -999, dtype=np.float64)
    print(f'event {i}: pT = {pt}, phi = {phi}, MET = {met}, MET_phi = {met_phi}')
    print(f'         Dphi_1 = {Dphi}')
    # if dphi > 20:
        # Dphi = 20 - dphi
    # else:
        # Dphi = dphi
    # Dphi[Dphi > 20] = Dphi[Dphi > 20] - 7
    Dphi[Dphi > 20] = Dphi[Dphi > 20] - 30
    min_Dphi = np.array([np.amin(Dphi)])
    print(f'         Dphi_2 = {Dphi}, min[Dphi] = {min_Dphi}')
    if Dphi.shape[0] < 4:
        diff = 4 - Dphi.shape[0]
        arr_n999 = np.full(diff, -999, dtype=np.float64)
        Dphi = np.concatenate((Dphi, arr_n999), axis=None)
    print(f'         Dphi_3 = {Dphi}')
    j_met = np.concatenate((j_met, Dphi[:4], min_Dphi), axis=None)
    print(f'         j_MET = {j_met}')
    _jet_met.append(j_met)
    print('-'*80)
np.stack(_jet_met, axis=0)

MET_phi = [(100.,  4.) (100.,  1.) (200.,  2.) (300., 30.)]
 --------------------------------------------------------------------------------
switch = False
There is NO jet.
event 0: pT = [], phi = [], MET = 100.0, MET_phi = 4.0
         Dphi_1 = [-999. -999. -999. -999.]
         Dphi_2 = [-999. -999. -999. -999.], min[Dphi] = [-999.]
         Dphi_3 = [-999. -999. -999. -999.]
         j_MET = [   0. -999. -999. -999. -999. -999.]
--------------------------------------------------------------------------------
switch = True
There is jet.
event 1: pT = [1 2], phi = [10 22], MET = 100.0, MET_phi = 1.0
         Dphi_1 = [ 9. 21.]
         Dphi_2 = [ 9. -9.], min[Dphi] = [-9.]
         Dphi_3 = [   9.   -9. -999. -999.]
         j_MET = [   2.    9.   -9. -999. -999.   -9.]
--------------------------------------------------------------------------------
switch = True
There is jet.
event 2: pT = [1 2 3], phi = [10 22 33], MET = 200.0, MET_phi = 2.0
         Dphi_1 = [ 8. 20. 31.]
        

array([[   0., -999., -999., -999., -999., -999.],
       [   2.,    9.,   -9., -999., -999.,   -9.],
       [   3.,    8.,   20.,    1., -999.,    1.],
       [   4.,   20.,    8.,    3.,    6.,    3.]])

### Test C

In [20]:
a = np.array([10, 11, 12, 20, 23])
print(a - 20)
np.absolute(a - 20)

[-10  -9  -8   0   3]


array([10,  9,  8,  0,  3])

In [21]:
# a = np.array([10, 11, 12, 20, 23])
a = np.array([])
print(f'1. {a[a > 11]}')
print('-'*20)

a[a > 11] = 7
print(f'2. {a}')

1. []
--------------------
2. []


In [22]:
a = np.array([10, 11, 12, 20, 23])
# a = np.array([])
a[a > 11] = a[a > 11] - 7
a

array([10, 11,  5, 13, 16])

In [23]:
print(np.min(a))
print(np.amin(a))

5
5


In [24]:
a = np.arange(4).reshape((2,2))
a

array([[0, 1],
       [2, 3]])

In [25]:
np.amin(a)

0

In [26]:
a = np.array([0])
print(a.shape)
a

(1,)


array([0])

In [27]:
a = np.array([])
print(a.shape)
a

(0,)


array([], dtype=float64)

In [28]:
np.amin(a)

ValueError: zero-size array to reduction operation minimum which has no identity

In [29]:
a - 5

array([], dtype=float64)